<a href="https://colab.research.google.com/github/nagayoshi3/deep_learning_practice/blob/master/3_2_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ニューラルネットワークによる翻訳
ニューラルネットワークによる翻訳を体験してみましょう。

## データの取得

In [0]:
!wget https://www.dropbox.com/s/d2bz2xmky5l9p8n/train.en?dl=0
!wget https://www.dropbox.com/s/ft08ydqr6msbsfo/train.ja?dl=0

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

def load_data(file_path):
    tokenizer = Tokenizer(filters="")
    whole_texts = []
    for line in open(file_path, encoding='utf-8'):
        whole_texts.append("<s> " + line.strip() + " </s>")
        
    tokenizer.fit_on_texts(whole_texts)
    
    return tokenizer.texts_to_sequences(whole_texts), tokenizer

# 読み込み＆Tokenizerによる数値化
x_train, tokenizer_en = load_data('train.en?dl=0')
y_train, tokenizer_ja = load_data('train.ja?dl=0')

show_ja=pd.read_csv('train.ja?dl=0', engine='python')
print(show_ja)

en_vocab_size = len(tokenizer_en.word_index) + 1
ja_vocab_size = len(tokenizer_ja.word_index) + 1

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.02, random_state=42)

# パディング
x_train = pad_sequences(x_train, padding='post')
y_train = pad_sequences(y_train, padding='post')

seqX_len = len(x_train[0])
seqY_len = len(y_train[0])

--2019-11-03 12:58:40--  https://www.dropbox.com/s/d2bz2xmky5l9p8n/train.en?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/d2bz2xmky5l9p8n/train.en [following]
--2019-11-03 12:58:40--  https://www.dropbox.com/s/raw/d2bz2xmky5l9p8n/train.en
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8c8a128b9cdd371e519cec727b.dl.dropboxusercontent.com/cd/0/inline/AroOa6Onz61dGCqe5mvbzb9t-wPiG2w_7sevVmkrNAVf4f8iadKblPLDMFKmSssQ8mgbT3rAvu5KzQW4NWT23DB8mb89t9Fi9y478ZwjX2my6g/file# [following]
--2019-11-03 12:58:41--  https://uc8c8a128b9cdd371e519cec727b.dl.dropboxusercontent.com/cd/0/inline/AroOa6Onz61dGCqe5mvbzb9t-wPiG2w_7sevVmkrNAVf4f8iadKblPLDMFKmSssQ8mgbT3rAvu5KzQW4NWT23DB8mb89t9Fi9y478ZwjX2my6g/file
Resolving uc8c8a128b9cdd37

        誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
0         多く の 動物 が 人間 に よ っ て 滅ぼ さ れ た 。
1                        私 は テニス 部員 で す 。
2                   エミ は 幸せ そう に 見え ま す 。
3           この 事実 を 心 に 留め て お い て 下さ い 。
4             彼女 は 私 たち の 世話 を し て くれ る 。
...                                   ...
49994             私 たち は その 結果 に 幻滅 し た 。
49995  私 たち は 英語 で 少な から ず 誤り を 犯 し ま す 。
49996              私 は 時間 を 持て余 し て い る 。
49997       私 たち と 外 へ 昼食 に い き ま せ ん か 。
49998               彼 は いらだ ち ながら 思 っ た 。

[49999 rows x 1 columns]


## モデルの構築

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Permute, Activation, Embedding, Dense, LSTM, concatenate, dot
from tensorflow.keras import backend as K

# エンコーダ
encoder_inputs = Input(shape=(seqX_len,))

emb_dim =256
encoder_embedded = Embedding(en_vocab_size, emb_dim, mask_zero=True)(encoder_inputs)
#　Embedding　単語を長さが一定のベクトルに変換します。
#emb_dim は出力ベクトルの次元数です。
                              
hid_dim = 256
encoded_seq, *encoder_states = LSTM(hid_dim, return_sequences=True, return_state=True)(encoder_embedded)
# Embeddingの出力を、LTSMの入力として利用します。その結果が、符号化器（エンコーダ）の出力になります。

# デコーダ
decoder_inputs = Input(shape=(seqY_len,))
decoder_embedding = Embedding(ja_vocab_size, emb_dim)
decoder_embedded = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(hid_dim, return_sequences=True, return_state=True)
decoded_seq, _, _ = decoder_lstm(decoder_embedded, initial_state=encoder_states)


# Attention
score_dense = Dense(hid_dim)
score = score_dense(decoded_seq)
score = dot([score, encoded_seq], axes=(2,2))
attention = Activation('softmax')(score)
context = dot([attention, encoded_seq], axes=(2,1))
concat = concatenate([context, decoded_seq], axis=2)

att_dim = 256
attention_dense = Dense(att_dim, activation='tanh')
attentional = attention_dense(concat)
output_dense = Dense(ja_vocab_size, activation='softmax')
outputs = output_dense(attentional)

model = Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## モデルの学習

In [0]:
import numpy as np

train_target = np.hstack((y_train[:, 1:], np.zeros((len(y_train),1), dtype=np.int32)))

model.fit([x_train, y_train], np.expand_dims(train_target, -1), batch_size=128, epochs=10, verbose=1, validation_split=0.2)

Train on 39200 samples, validate on 9800 samples
Epoch 1/10
39200/39200 [==============================] - 57s 1ms/sample - loss: 2.9385 - val_loss: 2.3038
Epoch 2/10
39200/39200 [==============================] - 52s 1ms/sample - loss: 2.1028 - val_loss: 1.9890
Epoch 3/10
39200/39200 [==============================] - 52s 1ms/sample - loss: 1.8443 - val_loss: 1.8104
Epoch 4/10
39200/39200 [==============================] - 52s 1ms/sample - loss: 1.6668 - val_loss: 1.6616
Epoch 5/10
39200/39200 [==============================] - 52s 1ms/sample - loss: 1.5219 - val_loss: 1.5368
Epoch 6/10
39200/39200 [==============================] - 52s 1ms/sample - loss: 1.3946 - val_loss: 1.4609
Epoch 7/10
39200/39200 [==============================] - 52s 1ms/sample - loss: 1.2828 - val_loss: 1.3794
Epoch 8/10
39200/39200 [==============================] - 52s 1ms/sample - loss: 1.1844 - val_loss: 1.3244
Epoch 9/10
39200/39200 [==============================] - 52s 1ms/sample - loss: 1.1007 - val_l

## モデルによる文章の生成

In [0]:
# エンコーダ（学習時と同じ構成、学習したレイヤーを利用）
encoder_model = Model(encoder_inputs, [encoded_seq]+encoder_states)

# デコーダ
decoder_states_inputs = [Input(shape=(hid_dim,)), Input(shape=(hid_dim,))]

decoder_inputs = Input(shape=(1,))
decoder_embedded = decoder_embedding(decoder_inputs)#学習済みEmbeddingレイヤーを利用
decoded_seq, *decoder_states = decoder_lstm(decoder_embedded, initial_state=decoder_states_inputs)# 学習済みLSTMレイヤーを利用

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoded_seq] + decoder_states)

# Attention
encoded_seq_in, decoded_seq_in = Input(shape=(seqX_len, hid_dim)), Input(shape=(1, hid_dim))
score = score_dense(decoded_seq_in)
score = dot([score, encoded_seq_in], axes=(2,2))
attention = Activation('softmax')(score)
context = dot([attention, encoded_seq_in], axes=(2,1))
concat = concatenate([context, decoded_seq_in], axis=2)
attentional = attention_dense(concat)
attention_outputs = output_dense(attentional)

attention_model = Model([encoded_seq_in, decoded_seq_in], [attention_outputs, attention])

def decode_sequence(input_seq, bos_eos, max_output_length = 1000):
    encoded_seq, *states_value = encoder_model.predict(input_seq)

    target_seq = np.array(bos_eos[0])  # bos_eos[0]="<s>"に対応するインデックス
    output_seq = bos_eos[0][:]
    attention_seq = np.empty((0,len(input_seq[0])))
    
    while True:
        decoded_seq, *states_value = decoder_model.predict([target_seq] + states_value)
        output_tokens, attention = attention_model.predict([encoded_seq, decoded_seq])
        sampled_token_index = [np.argmax(output_tokens[0, -1, :])]
        output_seq += sampled_token_index
        attention_seq = np.append(attention_seq, attention[0], axis=0)
        
        if (sampled_token_index == bos_eos[1] or len(output_seq) > max_output_length):
            break

        target_seq = np.array(sampled_token_index)

    return output_seq, attention_seq


%matplotlib inline 
import matplotlib.pyplot as plt

detokenizer_en = dict(map(reversed, tokenizer_en.word_index.items()))
detokenizer_ja = dict(map(reversed, tokenizer_ja.word_index.items()))

In [0]:
def evalation(text_no):
  input_seq = pad_sequences([x_test[text_no]], seqX_len, padding='post')
  bos_eos = tokenizer_ja.texts_to_sequences(["<s>", "</s>"])

  output_seq, attention_seq = decode_sequence(input_seq, bos_eos)

  print('元の文:', ' '.join([detokenizer_en[i] for i in x_test[text_no]]))
  print('生成文:', ' '.join([detokenizer_ja[i] for i in output_seq]))
  print('正解文:', ' '.join([detokenizer_ja[i] for i in y_test[text_no]]))

In [0]:
evalation(20)

元の文: <s> he is almost always at home . </s>
生成文: <s> 彼 は いつ も 家 に い る 。 </s>
正解文: <s> 彼 は ほとんど いつ も 家 に い る 。 </s>


In [0]:
evalation(2)

元の文: <s> i always brush my coat when i come home . </s>
生成文: <s> 私 は いつ も 帰 っ て くる の が いつ も 。 </s>
正解文: <s> 私 は 帰宅 する と いつ も コート に ブラシ を かけ る 。 </s>
